# Step-by-step manual :

## Building Mock maps :

### Mock spectra :

1. Run fake_spectra in parallel on TNG :


- First, we need to calculate required number of sightlines. It scales number of sightliens in LATIS to TNG300-1 cross section. The function below, takes the 3 data file from LATIS, which are the pixfile, mapfile and idsfile. 

The output is the number of random spectra we need to generate.

In [1]:
import codes.spectra_mocking as sm
sm.get_mock_sightline_number(pixfile='../spectra/maps/mapsv13/dcv13pix.dat', mapfile='../spectra/maps/mapsv13/dcv13map.dat',
                             idsfile='../spectra/maps/mapsv13/dcv13ids.dat', z_range=[2.4,2.6])

6738.573846169163

**Note:** These 3 observed data file are not publicly available yet. The only thing we need from those data are sightline densities at each redshift. So, you can skip the next cell and use the table below summarizing the sightline densities:

| z | required spectra in TNG300-1 |
| --| ---------------------------- |
| 2.3 | 9017 |
| 2.45 | 6739 |
| 2.6 | 4409 |

- Second we should set a small spectral resolution for our spectra, `res` argument in `fake_spectra`. 

  I have set it as   `res = detector resolution / 10`   , so later by averaging over the flux within 10 adjacent pixels, we'll get the spectra with detector's resolution. This is closer to what being **observationally measured** compared to summing up the optical depth across all pixels which would have been calculated if we had set the `res = detector resolution` . 
  

How to get detector and spectral resolutions ? They are the outputs here, respectively in the tuple :

In [2]:
sm.get_spec_res(z=2.4442257045541464)

(129.03335557027032, 147.6715069304205)

**Note :** For the cell above and for anything from here, you do not need the observed data.

Now, we pass the sightline number and resolution to `fake_Spectra.rand_spectra()`. There are some samples of submit files [here](https://github.com/mahdiqezlou/LyTomo-Watershed/tree/main/helper_scripts/generate_spectra) which call the helper code [here](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/helper_scripts/generate_spectra/parallel_spectra.py) to run fake_spectra in MPI.




2. Add noise and prepare for `dashchund` :

To add noise and write the spectra in a format readable to `dashchund`, run this code :

In [84]:
import importlib
importlib.reload(sm)
sm.write_input_dachshund_v2(spec_res=147.6715069304205, addpix=20, savefile='../spectra/spectra_TNG_z2.4_1.hdf5', 
                            output_file='../spectra/pix_TNG_z2.4_n1.dat', add_CE=True, add_CNR=True)

dvbin =  6.463172825361224
7832  sightlines. resolution:  6.463172825361224  z= 2.4442257045541464
dvbin =  6.463172825361224
7832  sightlines. resolution:  6.463172825361224  z= 2.4442257045541464
mean flux after noise = 0.8202996281424243
*** Error on mean flux :***  0.009905708551735004


### Run `dashchund` :

We have made dashchund output in last step. Now, we should make a config file which dashchund reads instruction from. That file looks like this, an explanation for each field is added after the # sign, remove them before running the dashchund :

```
lx = 205 # map size in cMpc/h
ly = 205
lz = 205
num_pixels = 1284579 # this is size of the pix file divided by 5
map_nx = 205  # number of pixels
map_ny = 205
map_nz = 205
corr_var_s = 0.05
corr_l_perp = 2.5
corr_l_para = 2.1
pcg_tol = 1.0e-3
pcg_max_iter = 100
pixel_data_path = ./pixel_files/pix_TNG_z2.4_averageF.dat
map_path = ./maps/map_TNG_z2.4.dat
```

Next we can run `dashchund` with : `dashchund.exe TNG_z2.4_n1_DCV13.cfg`. Or you can find a submission sample [here](https://github.com/mahdiqezlou/LyTomo-Watershed/helper_scripts/dachshund/submit) . 

## DM density field :

You need a dark matter density field on a regular grid for the next session when calculating the DM masses of the strucutres.

1. You can use the `TNG()` method [here](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/codes/get_density_field.py) to get that in an MPI fassion. The sample script to run with is [here](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/helper_scripts/DensityField/run_get_densityfield.py).

2. `TNG()` method writes down the outcome of each rank on an individual file, you need to use `LATIS.codes.get_density_field.make_full_mesh()` to add up the files. This is a serial code and already very quick.

## Strucutres:

Now, we need to find the strucutres within the mock map. For this, we need mock map, true map and DM density map. 

To find the optimal `peak threshold` value. Use the script [here](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/helper_scripts/watershed_params/thresh/get_th_parallel.py) to find these structures for a wide range of `peak threshold`. I run this MPI code in 3 chuncks of peak threshold ranges, $[2.0,2.25] - [2.3,2.55] - [2.6,2.85]$.


The outputs for each `peak threshold` are 2 files, one containing info such as tomography mass for all subregions and the other a file containing all contours defining the volume of subregions (label maps).

- To select the optimal watershed parametes, the island threshold significance, $\nu$, and the absorption peak significance, $\kappa$, look at [this notebook](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/select_kappa_nu.ipynb).

- To justify the selection of the parameters above and generally the watershed method, look at [this notebook](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/justify_watershed.ipynb)

- To provide an estimator on the Dark matter mass within these watersheds take a look at [this notebook](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/MDM_Mtomo.ipynb).

## Connection to z =0 :

In this step, we connect all structures to a FOF halo zt z=0. 


1.  we find all halos within each subregion. For this, use `highz_halos()` in `LATIS/codes/halos.py`. It is an MPI code, so you can run it using [this sample script](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/helper_scripts/descendants/run_find_halos.py) with a


2. Now we find the root subhalo descendant of the FirstSubhalo of all the halos within each subregion. It records lot's of info including the `SubhaloGrNr` which is the group number of the descendant subhalo. To do so, in `LATIS/codes/descendant.py` , we call `find_roots()`.  This code is MPI-parallel, you cnan use [this](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/helper_scripts/descendants/run_find_roots.py) smaple script to run it.


3. Now, all halos within a subregion vote for the FOF parent halos of the descendant subhalos . The votes are weighted by the mass of the halos at high-z. For this we use `find_voted_FOF_roots()` in `descendant.py`. This is also MPI-Parallel and you can use [this](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/helper_scripts/descendants/run_find_voted_FOF_roots.py) example script to run it.


- After creating this map between watersheds at z=2.5 and halos at z=0, take a look at [this notebook](https://github.com/mahdiqezlou/LyTomo-Watershed/blob/main/M0_Mtomo.ipynb) to provide an estimator for the descendant mass of each watershed. 
